In [1]:
import ast
import os
import time

import pandas as pd
import requests
from tqdm import tqdm

In [2]:
os.chdir("../../data/")

In [3]:
anime = pd.read_csv("AnimeList.csv")

In [4]:
def get_related_series(data, anime_id):
    df = pd.DataFrame({"source": [], "target": [], "relationship": []})
    related = ast.literal_eval(data)
    if not related:
        return df

    related_series = set()
    for key, val in related.items():
        for item in val:
            if item["type"] == "anime":
                df = df.append(
                    {"source": anime_id, "target": item["mal_id"], "relationship": key}
                    , ignore_index=True
                )
    return df

In [5]:
all_related_series = []
for i in tqdm(range(len(anime))):
    anime_id = anime.iloc[i]["anime_id"]
    related_series = get_related_series(anime.iloc[i]["related"], anime_id)
    all_related_series.append(related_series)
related_series_df = pd.concat(all_related_series, ignore_index=True)
related_series_df[['source', 'target']] = related_series_df[['source', 'target']].astype(int)

100%|██████████| 14478/14478 [00:40<00:00, 358.53it/s]


In [6]:
related_series_df.head()

,source,target,relationship
0,11013,13403,Sequel
1,2104,4163,Sequel
2,2104,5199,Side story
3,2104,28385,Alternative version
4,5262,2923,Prequel
